# Análisis de Sentimientos con VADER (Tweets) + Extensiones

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion1/7-sentiment-analysis.ipynb)

En este notebook realizamos un análisis de sentimientos binario (positivo vs negativo) usando el modelo VADER de NLTK, pero en lugar de reseñas de películas, usaremos un dataset diferente: tweets etiquetados (twitter_samples de NLTK).

### Referencias
* [Análisis de sentimiento de VADER](https://hex.tech/templates/sentiment-analysis/vader-sentiment-analysis/)

# Prerequisitos



In [1]:
# Instalación dependencias
import sys, pkgutil

def pip_install(pkgs):
    import subprocess, sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)

needed = ["pandas", "numpy", "scikit-learn", "matplotlib", "nltk"]
pip_install(needed)

print("Listo: dependencias instaladas.")


Listo: dependencias instaladas.


In [3]:
# Importaciones

import warnings
warnings.filterwarnings("ignore")

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, confusion_matrix, classification_report
)

SEED = 42
np.random.seed(SEED)

print("Imports OK.")


Imports OK.


# Dataset

Usaremos el corpus twitter_samples de NLTK, que trae:
- Tweets positivos
- Tweets negativos

Construimos un DataFrame con columnas:
- text: tweet
- label: pos o neg


In [4]:
import nltk
nltk.download("twitter_samples")

from nltk.corpus import twitter_samples

pos_tweets = twitter_samples.strings("positive_tweets.json")
neg_tweets = twitter_samples.strings("negative_tweets.json")

df_pos = pd.DataFrame({"text": pos_tweets, "label": "pos"})
df_neg = pd.DataFrame({"text": neg_tweets, "label": "neg"})
df = pd.concat([df_pos, df_neg], axis=0, ignore_index=True)

df.head(), df.label.value_counts()


[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


(                                                text label
 0  #FollowFriday @France_Inte @PKuchly57 @Milipol...   pos
 1  @Lamb2ja Hey James! How odd :/ Please call our...   pos
 2  @DespiteOfficial we had a listen last night :)...   pos
 3                               @97sides CONGRATS :)   pos
 4  yeaaaah yippppy!!!  my accnt verified rqst has...   pos,
 label
 pos    5000
 neg    5000
 Name: count, dtype: int64)

# Limpieza mínima de texto

En tweets es común encontrar:
- URLs
- menciones (@usuario)
- múltiples espacios

Haremos una limpieza suave (sin destruir emojis), ya que VADER usa señales como emoticonos y signos.


In [5]:
def clean_tweet(t: str) -> str:
    t = t.replace("\n", " ")
    t = re.sub(r"http\S+|www\.\S+", "", t)     # quitar URLs
    t = re.sub(r"@\w+", "", t)                # quitar menciones
    t = re.sub(r"\s+", " ", t).strip()        # normalizar espacios
    return t

df["text_clean"] = df["text"].astype(str).apply(clean_tweet)

# Remover vacíos (por seguridad)
df = df[df["text_clean"].str.len() > 0].reset_index(drop=True)

df[["text", "text_clean", "label"]].head()


,text,text_clean,label
0,#FollowFriday @France_Inte @PKuchly57 @Milipol...,#FollowFriday for being top engaged members in...,pos
1,@Lamb2ja Hey James! How odd :/ Please call our...,Hey James! How odd :/ Please call our Contact ...,pos
2,@DespiteOfficial we had a listen last night :)...,we had a listen last night :) As You Bleed is ...,pos
3,@97sides CONGRATS :),CONGRATS :),pos
4,yeaaaah yippppy!!! my accnt verified rqst has...,yeaaaah yippppy!!! my accnt verified rqst has ...,pos


#Separar en train/test

Aunque VADER no “entrena”, separar en test ayuda a:
- evaluar de forma más realista
- ajustar el umbral en train y reportar desempeño final en test


In [6]:
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=SEED, stratify=df["label"]
)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.label.value_counts(), test_df.label.value_counts()


(label
 neg    4000
 pos    4000
 Name: count, dtype: int64,
 label
 neg    1000
 pos    1000
 Name: count, dtype: int64)

# Línea base: VADER

En esta parte usamos VADER para obtener, para cada texto, cuatro puntajes: negativo, neutral, positivo y compound. El puntaje compound resume el sentimiento global en un solo número entre -1 y 1: valores cercanos a -1 indican más negatividad y valores cercanos a 1 indican más positividad.

Para convertir ese número en una etiqueta y poder evaluar el modelo, aplicamos una regla simple (línea base): si compound es mayor o igual a 0 lo marcamos como pos, y si es menor que 0 lo marcamos como neg. Esta regla es la misma idea del notebook guía, solo que aquí la aplicamos a nuestros datos.


In [7]:
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

def vader_compound(text: str) -> float:
    return sid.polarity_scores(text)["compound"]

train_df["compound"] = train_df["text_clean"].apply(vader_compound)
test_df["compound"]  = test_df["text_clean"].apply(vader_compound)

# Umbral baseline
baseline_threshold = 0.0
test_df["vader_pred_baseline"] = np.where(test_df["compound"] >= baseline_threshold, "pos", "neg")

test_df[["text_clean", "label", "compound", "vader_pred_baseline"]].head()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


,text_clean,label,compound,vader_pred_baseline
0,:(( but I want to get at least a little bit ol...,neg,0.8294,pos
1,that's all ok I know your busy :),pos,0.6369,pos
2,naeun :( body goals :(,neg,-0.7003,neg
3,"Ok,the first time we chat,and then i made such...",pos,0.4915,pos
4,everything okay? :( x,neg,-0.2500,neg
